In [1]:
import basemath as bm
import numpy as np

In [10]:
## Generating input data

avg_visitors = 250_000 # average number of visitors per day and variation
cr_A = 0.025 # conversion rate of control
cr_uplift = 0.01 # relative uplift of treatment
days = 60 # number of simulated days

num_visitors = np.abs(np.random.normal(avg_visitors - 1, avg_visitors * 0.1, days).astype(int)) + 1
customers_A = np.random.binomial(n=num_visitors, p=cr_A)
customers_B = np.random.binomial(n=num_visitors, p=cr_A*(1+cr_uplift))
customers_delta = customers_B - customers_A

In [14]:
## Running Basemath

# input parameters
mean_A = np.sum(customers_A) / np.sum(num_visitors)
mde = 0.01
alpha = 0.05
beta = 0.2
exp_name = "test_experiment_abc"

# initialise
bm_test = bm.BaseMathsTest(mean_A, mde, alpha, beta, seed = exp_name)

# day-by-day evaluation
experiment_state = 0
agg_visitors_seen_before = 0
agg_cust_delta_seen_before = 0
curr_day = 0

for day in range(days):
    experiment_state = bm_test.evaluate_experiment(
        agg_cust_delta_seen_before,
        customers_delta[day],
        agg_visitors_seen_before,
        num_visitors[day]
    )
    curr_day = day
    if experiment_state != 0:
        break

# summary
print(f"The experiment evaluation was stopped after {curr_day + 1} days.")
if experiment_state == 1:
    print("Treatment is significantly better than control.")
elif experiment_state == -1:
    print("Treatment is NOT significantly better than control.")
elif experiment_state == 0:
    print("Experiment stopped without conclusion. Not enough data.")
    print(f"{np.sum(num_visitors)} data points per day provided, {bm_test.required_samples} required")

The experiment evaluation was stopped after 60 days.
Experiment stopped without conclusion. Not enough data.
14981579 data points per day provided, 5426125 required


In [37]:
print(agg_cust_delta_seen_before,
        customers_delta[day],
        agg_visitors_seen_before,
        num_visitors[day])

0 112 0 252580


In [45]:
bm_test.evaluate_experiment(0, 100, 0, 250000)

TypeError: evaluate_experiment() takes 4 positional arguments but 5 were given

In [46]:
import inspect
print(inspect.signature(bm_test.evaluate_experiment))

(val_A, val_B, batch_samples)
